The same as before.

In [1]:
import pandas as pd
import Utils.dataset_processing as dp

sup_data_path = 'Data/train_data_labeled.jsonl'
Super_datatable = dp.read_json_to_table(sup_data_path)

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

In [3]:
superdata_loader = dp.dataset_loader(Super_datatable, tokenizer)

In [4]:
from torch.utils.data import random_split

data1, data2 = random_split(superdata_loader, [0.7, 0.3])

In [5]:
data1_loader = dp.load_data(data1, 12)
data2_loader = dp.load_data(data2, 16)
full_loader = dp.load_data(superdata_loader, 16)

In [6]:
import robust_loss
import model
import torch
import Utils.train as tr
import torch.nn as nn

model = model.RoBERTa_Classify("roberta-large", 3)
model.load_state_dict(torch.load("_PFTed.model"))

model_stad = model.state_dict()
for name, para in model.named_parameters(): 
    model_stad[name][:] += (torch.rand(para.size())-0.5) * 0.05 * torch.std(para)
model.load_state_dict(model_stad)
model.cuda()

newloss = nn.CrossEntropyLoss(label_smoothing = 0.1)
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = 1.87e-5, weight_decay = 1.12e-5)
model.compile(opt, newloss)

tr.train_classify_model(
    model,
    7,
#    data1_loader,
#    data2_loader,
    data1_loader,
    data2_loader,
    save_best = 'acc'
) 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch   1 /   7


  0%|0/46[00:00<?]

val_loss:0.7775, val_acc:0.7585
>>>>>>>>>>>>>>>>>>>>>>>Best result:0.7585>>>>>>>>>>>>>>>>>>>>>>>
Epoch   2 /   7


  0%|0/46[00:00<?]

...

val_loss:0.7313, val_acc:0.7839
Epoch   6 /   7


  0%|0/46[00:00<?]

val_loss:0.8985, val_acc:0.7076
Epoch   7 /   7


  0%|0/46[00:00<?]

val_loss:0.8919, val_acc:0.7119


In [7]:
torch.save(model.state_dict(), "_fine_tuned_4.model")

In [8]:
import numpy as np
import pickle

seed = np.random.get_state()

with open("_step3_4_randomseed.pk", "wb") as file:
    pickle.dump(seed, file)